<a href="https://colab.research.google.com/github/DFwJZ/Real-time_Bot_Detection/blob/main/Cresci2017_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Meagan's file path
root_path = "/content/drive/MyDrive/Bot_detection/cresci tweets/cresci 2017"

In [ ]:
# Jason's file path
root_path = "/content/drive/MyDrive/Colab Notebooks/Summer_Camp/Bot_detection/cresci tweets/cresci 2017"
# Jason's school email file path
# root_path = "/content/drive/MyDrive/Bot_detection/cresci tweets/cresci 2017"


In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
# Genuine Data Read in
try:
    df_genuine_data = pd.read_csv(f'{root_path}/genuine_tweets.csv',encoding = 'latin_1')
    print("Data loaded successfully with 'latin_1' encoding.")
except Exception as e:
    print("Error while reading the CSV file:", e)

<ipython-input-20-6460d800ed46>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_genuine_data = pd.read_csv(f'{root_path}/genuine_tweets.csv',encoding = 'latin_1')


Data loaded successfully with 'latin_1' encoding.


In [ ]:
df_genuine_data = df_genuine_data.iloc[:, :-2]

In [ ]:
df_genuine_data.shape

(2839362, 23)

In [ ]:
expected_columns_genuine = df_genuine_data.columns.tolist()
expected_columns_genuine

['id',
 'text',
 'source',
 'user_id',
 'truncated',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'in_reply_to_screen_name',
 'retweeted_status_id',
 'geo',
 'place',
 'contributors',
 'retweet_count',
 'reply_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'possibly_sensitive',
 'num_hashtags',
 'num_urls',
 'num_mentions',
 'created_at',
 'timestamp']

In [ ]:
# Read fake data
fake_data_file =['2017_fake_followers_tweets.csv', '2017_spam1_tweets.csv', '2017_social1_tweets.csv', '2017_social2_tweets.csv', '2017_social3_tweets.csv']

df_fake_combined = pd.DataFrame()
common_columns = None

# Load the fake data
for file in fake_data_file:
    try:
        df = pd.read_csv(f'{root_path}/{file}', encoding='latin_1', low_memory=False)

        if df_fake_combined.empty:
            # This is the first file - initialize df_combined and common_columns
            df_fake_combined = df
            common_columns = set(df.columns)
        else:
            # This is a subsequent file - update common_columns and append only common columns to df_combined
            common_columns &= set(df.columns)
            df_fake_combined = pd.concat([df_fake_combined, df[list(common_columns)]])

        print(f"{file} loaded successfully with 'latin_1' encoding.")
    except Exception as e:
        print(f"Error while reading the {file} CSV file:", e)

2017_fake_followers_tweets.csv loaded successfully with 'latin_1' encoding.
2017_spam1_tweets.csv loaded successfully with 'latin_1' encoding.
2017_social1_tweets.csv loaded successfully with 'latin_1' encoding.
2017_social2_tweets.csv loaded successfully with 'latin_1' encoding.
2017_social3_tweets.csv loaded successfully with 'latin_1' encoding.


In [ ]:
df_fake_combined.shape

(3798254, 23)

In [ ]:
from sklearn.model_selection import train_test_split
# Train_fake_df will be used to train VAE to get baseline model
train_fake_df, test_fake_df = train_test_split(df_fake_combined, test_size=0.1, random_state=719)
_, test_genuine_df = train_test_split(df_genuine_data, test_size=0.1, random_state=719)

In [ ]:
# test_fake_df['bot'] = 1
# test_genuine_df['bot'] = 0

In [ ]:
# df_test_combined = pd.concat([test_fake_df, test_genuine_df])

In [ ]:
# df_test_combined.head()

In [ ]:
# roughly 85% to 15%
train_fake_df.shape, test_fake_df.shape, test_genuine_df.shape

((3418428, 23), (379826, 23), (283937, 23))

### Generate training data-DistilBERT


In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import tqdm
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
class DistilBERT:
    def __init__(self, batch_size=32):
        """ Initialize Distil-BERT model """
        self.model_name = 'distilbert-base-cased'
        self.model_class = DistilBertModel
        self.tokenizer_class = DistilBertTokenizer

        self.tokenizer = self.tokenizer_class.from_pretrained(self.model_name)
        self.model = self.model_class.from_pretrained(self.model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)

        self.batch_size=batch_size

    def embed(self, samples):
        n = len(samples)
        features = []

        for i in range(0, n, self.batch_size):
            batch = samples[i:i+self.batch_size]
            encoded_input = self.tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
            encoded_input = {key: value.to(self.device) for key, value in encoded_input.items()}

            with torch.no_grad():
                last_hidden_states = self.model(**encoded_input)

            batch_features = last_hidden_states.last_hidden_state[:, 0, :].cpu().numpy()
            features.extend(batch_features)

        return np.array(features)

In [ ]:
distil_bert = DistilBERT(batch_size=512)
print("Success!") if distil_bert else print("Fail!")

Success!


In [ ]:
# tqdm library is used to display the process bar (1s interval), easy to track the process when running in backgroud,
tqdm.tqdm.pandas(mininterval=1)

In [ ]:
data_training_text= train_fake_df.dropna(subset=['text'])
data_training_text.shape

(3418039, 23)

In [ ]:
test_fake_df = test_fake_df.dropna(subset=['text'])
test_fake_df.shape

(379780, 23)

In [ ]:
test_genuine_df = test_genuine_df.dropna(subset=['text'])
test_genuine_df.shape

(282663, 23)

In [ ]:
# df_test_combined = df_test_combined.dropna(subset=['text'])

# data_test_text= df_test_combined[['text']]
# data_test_label = df_test_combined[['bot']]
# data_test_text.shape, data_test_label.shape

((662443, 1), (662443, 1))

In [ ]:
if os.path.exists(f'{root_path}/Autoencoder'):
    print('yes')

yes


In [ ]:
data_training_text_only_full = data_training_text['text']
data_test_fake_df_full = test_fake_df['text']
data_test_genuine_df_full = test_genuine_df['text']
data_training_text_only_full.shape, data_test_fake_df_full.shape, data_test_genuine_df_full.shape

((3418039,), (379780,), (282663,))

In [ ]:
data_training_text_only_full.to_csv(f'{root_path}/Autoencoder/data_training_text_only_full.csv', index=False)
data_test_fake_df_full.to_csv(f'{root_path}/Autoencoder/data_test_fake_df_full.csv', index=False)
data_test_genuine_df_full.to_csv(f'{root_path}/Autoencoder/data_test_genuine_df_full.csv', index=False)

In [ ]:
data_training_text_only_full = pd.read_csv(f'{root_path}/Autoencoder/data_training_text_only_full.csv', encoding = 'latin_1')
# data_test_fake_df_full = pd.read_csv(f'{root_path}/Autoencoder/data_test_fake_df_full.csv', encoding = 'latin_1')
# data_test_genuine_df_full = pd.read_csv(f'{root_path}/Autoencoder/data_test_genuine_df_full.csv', encoding = 'latin_1')

In [ ]:
data_training_text_only_full.shape, data_test_fake_df_full.shape, data_test_genuine_df_full.shape

In [ ]:
# Shuffle the data
data_training_text_only_full = data_training_text_only_full.sample(frac=1, random_state=720)

In [ ]:
data_training_text_only_400k = data_training_text_only_full[:400000]
data_training_text_only_400k

,text
0,Se non avete mai rischiato di soffocare tentan...
1,Check out these awesome ping pong &amp; table ...
2,TIP: Trash containers should always be covered...
3,MAMMA MAMMA E' ARRIVATO IL SALUMIERE CON LA SP...
4,Tutti che continua a regalarmi libri e nessuno...
...,...
399995,RITWITTA Se anche anche a te ÃÂ¨ capitato di ...
399996,Le persone che quando dicono una barzelletta n...
399997,La vera felicitÃÂ ÃÂ¨ la pace con se stessi...
399998,"L'AMORE TI FA' FARE COSE PAZZE, AD ESEMPIO IO...."


In [ ]:
# # Function to process texts in batch
# def process_texts(data, distil_bert):
#     embeddings = []
#     data = data['text'].tolist()  # Convert the Series to a list
#     for i in tqdm.tqdm(range(0, len(data), distil_bert.batch_size), total=int(len(data) / distil_bert.batch_size), desc='Processing'):
#         batch_texts = data[i:i+distil_bert.batch_size]
#         batch_embedding = distil_bert.embed(batch_texts)
#         embeddings.append(batch_embedding)
#     return np.concatenate(embeddings)


# data_training_text_only_full_embeddings = process_texts(data_training_text_only_400k, distil_bert)
# data_test_fake_df_full_embeddings = process_texts(data_test_fake_df_full, distil_bert)
# data_test_genuine_df_full_embeddings = process_texts(data_test_genuine_df_full, distil_bert)

# np.save(f'{root_path}/Autoencoder/data_training_text_embeddings_400k.npy', data_training_text_only_full_embeddings)
# np.save(f'{root_path}/Autoencoder/data_test_fake_df_full_embeddings.npy', data_test_fake_df_full_embeddings)
# np.save(f'{root_path}/Autoencoder/data_test_genuine_df_full_embeddings.npy', data_test_genuine_df_full_embeddings)

Processing: 100%|██████████| 12500/12500 [14:47<00:00, 14.08it/s]
Processing: 11869it [13:52, 14.25it/s]                           
Processing: 8834it [09:41, 15.18it/s]                          


In [ ]:
chunk_size = 500000 # 0.5 million
num_chunks = len(data_training_text_only_full) // chunk_size + 1

def process_texts(data, distil_bert):
    embeddings = []
    data = data['text'].tolist()  # Convert the Series to a list
    for i in tqdm.tqdm(range(0, len(data), distil_bert.batch_size), total=int(len(data) / distil_bert.batch_size), desc='Processing'):
        batch_texts = data[i:i+distil_bert.batch_size]
        batch_embedding = distil_bert.embed(batch_texts)
        embeddings.append(batch_embedding)
    return np.concatenate(embeddings)

for i in range(num_chunks):
    start_index = i * chunk_size
    end_index = min((i + 1) * chunk_size, len(data_training_text_only_full))
    data_chunk = data_training_text_only_full[start_index:end_index]

    # Define the path where you want to save the numpy array
    npy_path = f'{root_path}/Autoencoder/data_training_text_embeddings_chunk_{i}.npy'

    # Check if this chunk has already been processed and saved
    if os.path.exists(npy_path):
        print(f'Chunk {i} has already been processed.')
    else:
        # Process this chunk of data
        data_chunk_embeddings = process_texts(data_chunk, distil_bert)

        # Save the processed data as a numpy array
        np.save(npy_path, data_chunk_embeddings)


In [ ]:
data_training_text_embeddings_chunk_0 = np.load(f'{root_path}/Autoencoder/data_training_text_embeddings_chunk_0.npy')
data_training_text_embeddings_chunk_1 = np.load(f'{root_path}/Autoencoder/data_training_text_embeddings_chunk_1.npy')

In [ ]:
data_training_text_embeddings_chunk_0_n_1 = np.concatenate((data_training_text_embeddings_chunk_0,data_training_text_embeddings_chunk_1), axis=0)
np.random.shuffle(data_training_text_embeddings_chunk_0_n_1)
data_training_text_embeddings_500k = data_training_text_embeddings_chunk_0_n_1[:500000]
np.save(f'{root_path}/Autoencoder/data_training_text_embeddings_500k.npy', data_training_text_embeddings_500k)

### Autoencoder impl

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, ReLU, LeakyReLU, BatchNormalization, \
     Flatten, Dense, Reshape, Conv2DTranspose, Activation, Lambda, Dropout
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.losses import mse
from tensorflow.python.framework.ops import disable_eager_execution
import numpy as np

In [ ]:
class KLDivergenceLayer(tf.keras.layers.Layer):
    """ Layer to add KL divergence to the model loss. """
    def __init__(self, *args, **kwargs):
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):
        mu, log_variance = inputs
        kl_loss = -0.5 * K.mean(K.sum(1 + log_variance - K.square(mu) - K.exp(log_variance), axis=-1))
        self.add_loss(kl_loss)
        return inputs

class VAE_:
    """ VAE represents a Deep Variational Autoencoder architecture with mirrored encoder and decoder components. """
    def __init__(self, input_shape, intermediate_dim, latent_space_dim, dropout_rate=0.3):
        self.input_shape = input_shape  # (768,)
        self.intermediate_dim = intermediate_dim  # e.g., 256
        self.latent_space_dim = latent_space_dim  # e.g., 2
        self.dropout_rate = dropout_rate

        self.encoder = None
        self.decoder = None
        self.model = None

        self._build()

    def _build(self):
        self._build_encoder()
        self._build_decoder()
        self._build_autoencoder()

    def _build_encoder(self):
        encoder_input = Input(shape=self.input_shape, name="encoder_input")
        x = Dense(self.intermediate_dim, activation='relu')(encoder_input)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout_rate)(x)
        # x = Dense(self.intermediate_dim // 2, activation='relu')(x)  # Additional layer
        # x = BatchNormalization()(x)
        # x = Dropout(self.dropout_rate)(x)

        mu = Dense(self.latent_space_dim, name="mu")(x)
        log_variance = Dense(self.latent_space_dim, name="log_variance")(x)

        kl_layer = KLDivergenceLayer()([mu, log_variance])

        def sample_point_from_normal_distribution(args):
            mu, log_variance = args
            epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
            sampled_point = mu + K.exp(log_variance / 2) * epsilon
            return sampled_point

        sampled_point = Lambda(sample_point_from_normal_distribution, name="encoder_output")(kl_layer)

        self.encoder = Model(encoder_input, sampled_point, name="encoder")

    def _build_decoder(self):
        decoder_input = Input(shape=(self.latent_space_dim,), name="decoder_input")
        x = Dense(self.intermediate_dim, activation='relu')(decoder_input)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout_rate)(x)
        # x = Dense(self.intermediate_dim, activation='relu')(x)
        # x = BatchNormalization()(x)
        # x = Dropout(self.dropout_rate)(x)
        decoder_output = Dense(np.prod(self.input_shape), activation='sigmoid')(x)

        self.decoder = Model(decoder_input, decoder_output, name="decoder")

    def _build_autoencoder(self):
        autoencoder_input = Input(shape=self.input_shape, name="autoencoder_input")
        encoded_img = self.encoder(autoencoder_input)
        decoded_img = self.decoder(encoded_img)
        self.model = Model(autoencoder_input, decoded_img, name="autoencoder")

    def compile(self, learning_rate=0.0001):
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    def train(self, x_train, batch_size, num_epochs):
        self.model.fit(x_train, x_train, batch_size=batch_size, epochs=num_epochs, shuffle=True)

    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        self.model.summary()


In [ ]:
# # disable_eager_execution()

# # Doesn't have to be a method
# def _calculate_reconstruction_loss(y_target, y_predicted):
#     reconstruction_loss = mse(y_target, y_predicted)
#     return reconstruction_loss

# #Change properly
# def calculate_kl_loss(model):
#     # wrap `_calculate_kl_loss` such that it takes the model as an argument,
#     # returns a function which can take arbitrary number of arguments
#     # (for compatibility with `metrics` and utility in the loss function)
#     # and returns the kl loss
#     def _calculate_kl_loss(*args):
#         kl_loss = -0.5 * K.sum(1 + model.log_variance - K.square(model.mu) -
#                               K.exp(model.log_variance), axis=1)
#         return kl_loss
#     return _calculate_kl_loss

# class VAE:
#     """
#     VAE represents a Deep Variational Autoencoder architecture with mirrored encoder and decoder components.
#     """

#     def __init__(self, input_shape, intermediate_dim, latent_space_dim):
#         self.input_shape = input_shape  # (768,)
#         self.intermediate_dim = intermediate_dim  # e.g., 256
#         self.latent_space_dim = latent_space_dim  # e.g., 10

#         self.encoder = None
#         self.decoder = None
#         self.model = None

#         self.mu = None  # Mean
#         self.log_variance = None  # Variance

#         self._build()

#     def _build(self):
#         self._build_encoder()
#         self._build_decoder()
#         self._build_autoencoder()

#     def _build_encoder(self):
#         encoder_input = Input(shape=self.input_shape, name="encoder_input")
#         x = Dense(self.intermediate_dim, activation='relu')(encoder_input)

#         self.mu = Dense(self.latent_space_dim, name="mu")(x)
#         self.log_variance = Dense(self.latent_space_dim, name="log_variance")(x)

#         def sample_point_from_normal_distribution(args):
#             mu, log_variance = args
#             epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
#             sampled_point = mu + K.exp(log_variance / 2) * epsilon
#             return sampled_point

#         encoder_output = Lambda(sample_point_from_normal_distribution, name="encoder_output")([self.mu, self.log_variance])

#         self.encoder = Model(encoder_input, encoder_output, name="encoder")

#     def _build_decoder(self):
#         decoder_input = Input(shape=(self.latent_space_dim,), name="decoder_input")
#         x = Dense(self.intermediate_dim, activation='relu')(decoder_input)
#         decoder_output = Dense(np.prod(self.input_shape), activation='sigmoid')(x)

#         self.decoder = Model(decoder_input, decoder_output, name="decoder")

#     def _build_autoencoder(self):
#         autoencoder_input = Input(shape=self.input_shape, name="autoencoder_input")
#         encoded_img = self.encoder(autoencoder_input)
#         decoded_img = self.decoder(encoded_img)
#         self.model = Model(autoencoder_input, decoded_img, name="autoencoder")

#     def compile(self, learning_rate=0.0001):
#         # optimizer = Adam(learning_rate=learning_rate)
#         self.model.compile(tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate),
#                            loss=self._calculate_combined_loss,
#                            metrics=[_calculate_reconstruction_loss,
#                                     calculate_kl_loss(self)])

#     def _calculate_combined_loss(self, y_target, y_predicted):
#         reconstruction_loss = _calculate_reconstruction_loss(y_target, y_predicted)
#         kl_loss = calculate_kl_loss(self)()
#         combined_loss = reconstruction_loss + kl_loss
#         return combined_loss

#     def train(self, x_train, batch_size, num_epochs):
#         self.model.fit(x_train, x_train, batch_size=batch_size, epochs=num_epochs, shuffle=True)

#     def summary(self):
#         self.encoder.summary()
#         self.decoder.summary()
#         self.model.summary()

In [ ]:

# autoencoder = VAE(
#     input_shape = 768,
#     intermediate_dim = 128,  # e.g., 128
#     latent_space_dim = 3  # e.g., 3
# )
# autoencoder.summary()

In [ ]:
INPUT_SHAPE = (768,)
INTERMEDIATE_DIM = 256
LATENT_SPACE_DIM = 10

embeddings_path = f'{root_path}/Autoencoder/data_training_text_embeddings_500k.npy'

def load_embeddings(embeddings_path):
    embeddings = np.load(embeddings_path)
    return embeddings

def train(x_train, learning_rate, batch_size, epochs):
    autoencoder = VAE_(
        input_shape=INPUT_SHAPE,
        intermediate_dim=INTERMEDIATE_DIM,
        latent_space_dim=LATENT_SPACE_DIM
    )
    # autoencoder.summary()
    autoencoder.compile(learning_rate)
    autoencoder.train(x_train, batch_size, epochs)
    return autoencoder


In [ ]:
x_train = load_embeddings(embeddings_path)

In [ ]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 8
EPOCHS = 2

autoencoder = train(x_train, LEARNING_RATE, BATCH_SIZE, EPOCHS)


Epoch 1/2
62500/62500 [==============================] - 325s 5ms/step - loss: 0.3574
Epoch 2/2
62500/62500 [==============================] - 326s 5ms/step - loss: 0.2195


In [ ]:
# autoencoder.model.save(f'{root_path}/Autoencoder/VAE_model_batch16_epo2.h5')
autoencoder.model.save(f'{root_path}/Autoencoder/VAE_model_batch16_epo2_721.h5')

In [ ]:
from tensorflow.keras.models import load_model
autoencoder = load_model(f'{root_path}/Autoencoder/VAE_model_batch16_epo2_721.h5', custom_objects={'KLDivergenceLayer': KLDivergenceLayer})

### Reconstruction


In [ ]:
# data_training_text_embeddings_400k = np.load(f'{root_path}/Autoencoder/data_training_text_embeddings_400k.npy')
data_training_text_embeddings_500k = np.load(f'{root_path}/Autoencoder/data_training_text_embeddings_500k.npy')


In [ ]:
# Reconstruction for training data
x_train_pred = autoencoder.predict(data_training_text_embeddings_500k)
np.save(f'{root_path}/Autoencoder/data_train_predicted_500k.npy', x_train_pred)

15625/15625 [==============================] - 43s 2ms/step


In [ ]:
mae_vector = np.mean(np.power(x_train_pred - data_training_text_embeddings_500k, 2), axis=1)
error_thresh = np.quantile(mae_vector, 0.99)
error_thresh

0.23986363545060158

In [ ]:
def get_error_term(pred, actual):
    """Calculate the Mean Squared Error between predicted and actual values"""
    mae = np.mean(np.power(pred - actual, 2), axis=1)
    return mae

In [ ]:
data_test_fake_df_full_embeddings = np.load(f'{root_path}/Autoencoder/data_test_fake_df_full_embeddings.npy')


In [ ]:
data_test_genuine_df_full_embeddings = np.load(f'{root_path}/Autoencoder/data_test_genuine_df_full_embeddings.npy')

In [ ]:
error_thresh = 0.22

In [ ]:
chunk_size = 100000  # Adjust this to fit your memory constraints
num_anomalies = 0
for i in range(0, len(data_test_fake_df_full_embeddings), chunk_size):
    chunk = data_test_fake_df_full_embeddings[i:i+chunk_size]
    chunk_pred = autoencoder.predict(chunk)
    mae_chunk = get_error_term(chunk_pred, chunk)
    anomalies_chunk = (mae_chunk > error_thresh)
    num_anomalies += np.sum(anomalies_chunk)

print(f"Number of anomalies in fake data: {num_anomalies}")

2494/2494 [==============================] - 5s 2ms/step
Number of anomalies in fake data: 169197


In [ ]:
chunk_size = 100000  # Adjust this to fit your memory constraints
num_anomalies = 0
for i in range(0, len(data_test_genuine_df_full_embeddings), chunk_size):
    chunk = data_test_genuine_df_full_embeddings[i:i+chunk_size]
    chunk_pred = autoencoder.predict(chunk)
    mae_chunk = get_error_term(chunk_pred, chunk)
    anomalies_chunk = (mae_chunk > error_thresh)
    num_anomalies += np.sum(anomalies_chunk)

print(f"Number of anomalies in genuine data: {num_anomalies}")

2584/2584 [==============================] - 6s 2ms/step
Number of anomalies in genuine data: 80713


In [ ]:
from sklearn.decomposition import

PCAX_encoded = encoder.predict(X_test)
pca = PCA(n_components=2)
X_transform = pca.fit_transform(X_encoded)